# Création du Dataframe + initialisation de la Data

Pensez a gerer les colonnes en trop excel (df.drop) \
Pour chimiste 2 mso trop long penser a tout mettre dans 1 case \
A faire je pense gestion du profile du gars du type bouton `Mon Profil` avec possibilité de changer année groupe filiere et Mso

In [9]:
import pandas as pd
import json
from numpyencoder import NumpyEncoder

# Read csv file
date = "01_04"
Df = pd.read_csv(f'./planning_Csv/4ETI/{date}.csv', sep=';')
# Filter columns and rows
Df.reset_index()
Df.fillna(0, inplace=True)
# on ne garde que les 5 premieres columns
Df = Df.iloc[:, 0:5]
Df

FileNotFoundError: [Errno 2] No such file or directory: 'planning_Csv\\4ETI\\01_04.csv'

### On récupère la liste de toutes les matière/prof/majeurs dans LstSemaine

In [7]:
def createSchedule(Df):
    DicoJour = {}
    IsAprem = False
    LstSemaine = []
    # parcour des jours pour séparer matin et aprem
    for obj in Df :
        LstMatin = []
        LstAprem = []
        LstTemp = []
        # parcour des cases non nulles 
        for case in Df[obj] :
            if case != 0 :
                LstTemp.append(case.strip())

        # parcour des cases non nulles d'une journée pour la séparation
        for index,value in enumerate(LstTemp) :
            if index == 0 :
                # on ajoute le jour
                DicoJour["jour"] = obj +"-" + value
            else :
                if value in {'13h-14h30','13h30-17h45','13h10-18h', '13h30-15h30', '15h45-17h45', '13h30 - 14h30','13h30 - 17h45','13h30 - 17h30'} :
                  IsAprem = True
                elif value in {'8h-12h15','8h - 12h15', '8h30 - 10h00', '8h-10h', '10h15-12h15','10h15 - 12h15', '8h- 10h'}:
                    IsAprem = False
                if "\n" in value :
                    # on ajoute les cours
                    if IsAprem :
                        LstAprem.append(value.split("\n")[0])
                        LstAprem.append(value.split("\n")[1])
                    else :
                        LstMatin.append(value.split("\n")[0])
                        LstMatin.append(value.split("\n")[1])
                    continue
                if IsAprem :
                    LstAprem.append(value)
                else :
                    LstMatin.append(value)

        DicoJour["Matin"] = LstMatin.copy()
        DicoJour["Aprem"] = LstAprem.copy()
        LstSemaine.append(DicoJour.copy())
    return LstSemaine

LstSemaine = createSchedule(Df)
df_semaine = pd.DataFrame( LstSemaine)
df_semaine
#df_semaine.to_json('planning_semaineMatin_Aprem_TEST.json',orient='records', indent=4)

,jour,Matin,Aprem
0,LUNDI-27-mars,"[8h30-12h30, CMH, 8h30-10h TD surdité, 10h15 1...","[13h10-18h, LV, (Voir planning)]"
1,MARDI-28-mars,"[8h-12h15, GROUPE B, Projet, F. JOLY, Salle I2...","[13h30-17h45, GROUPE C&D, Projet, F. JOLY, Sal..."
2,MERCREDI-29-mars,"[8h-12h15, GROUPE A & B, Projet, F. JOLY, Sall...",[]
3,JEUDI-30-mars,"[8h-12h15, GROUPE D, Projet, F. JOLY, Salle i2...",[13h30-17h45]
4,VENDREDI-31-mars,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, Pas de ...","[13h30-17h45, MSO1 OUVERTURE, AWS Cloud Founda..."


### On filtre les Majeurs et les GROUPES par jour/ Demi-journée et majeurs ou tronc commun 4ETI

In [8]:
import copy
def filtreMajGrp(pLstSemaine):
    Semaine =  {}
    lstGroupesinit = ['A','B','C','D','TousLesGroupes']
    lstMajeursinit = ['CONCEP.LOGICIELLE/BIG DATA','ROBOTIQUE','ELECTRONIQUE ET SYST EMB','INFRA DES RESEAUX','IMAGE','Pour tous']

    # creation dico {grp: [], ...}
    dicGrpinit = {}
    for grp in lstGroupesinit :
        dicGrpinit[grp] = []
    
    # creation dico {majeur: {grp: [], ...}, ...}
    dicMajinit = {}
    for maj in lstMajeursinit :
        dicMajinit[maj] = copy.deepcopy(dicGrpinit)
    
    print(f'{dicMajinit=}')

    print(f"test: {id(dicMajinit['ROBOTIQUE']['A'])=}", f"and {id(dicMajinit['IMAGE']['A'])=}")

    # parcours des jours de la semaine
    for jour in pLstSemaine :
        dicMaj = copy.deepcopy(dicMajinit)
        print(f'{jour["jour"]=}')
        Majeur = "Pour tous" #on initialise le majeur à tous
        grp = "TousLesGroupes" #on initialise le groupe à tous
        # creation de la journee ("LUNDI-..." : {})
        Semaine[jour["jour"]] = {}

        # parcours de la matinée et on les ajoute au majeur correspondant 
        for Case in jour["Matin"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            elif Case in lstGroupesinit:
                grp = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur][grp].append(Case)
        Semaine[jour["jour"]]["Matin"] = copy.deepcopy(dicMaj)

        # parcours de l'aprem et on les ajoute au majeur correspondant
        dicMaj = copy.deepcopy(dicMajinit)
        for Case in jour["Aprem"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            elif Case in lstGroupesinit:
                grp = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur][grp].append(Case)
        Semaine[jour["jour"]]["Aprem"] = copy.deepcopy(dicMaj)

    print('fin')
    return Semaine

PlanningMajeurGrp = filtreMajGrp(copy.deepcopy(LstSemaine)) # KK
df_planningMajGrp = pd.DataFrame(PlanningMajeurGrp)
df_planningMajGrp

dicMajinit={'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': [], 'C': [], 'D': [], 'TousLesGroupes': []}, 'ROBOTIQUE': {'A': [], 'B': [], 'C': [], 'D': [], 'TousLesGroupes': []}, 'ELECTRONIQUE ET SYST EMB': {'A': [], 'B': [], 'C': [], 'D': [], 'TousLesGroupes': []}, 'INFRA DES RESEAUX': {'A': [], 'B': [], 'C': [], 'D': [], 'TousLesGroupes': []}, 'IMAGE': {'A': [], 'B': [], 'C': [], 'D': [], 'TousLesGroupes': []}, 'Pour tous': {'A': [], 'B': [], 'C': [], 'D': [], 'TousLesGroupes': []}}
test: id(dicMajinit['ROBOTIQUE']['A'])=4827845248 and id(dicMajinit['IMAGE']['A'])=4827846208
jour["jour"]='LUNDI-27-mars'
jour["jour"]='MARDI-28-mars'
jour["jour"]='MERCREDI-29-mars'
jour["jour"]='JEUDI-30-mars'
jour["jour"]='VENDREDI-31-mars'
fin


,LUNDI-27-mars,MARDI-28-mars,MERCREDI-29-mars,JEUDI-30-mars,VENDREDI-31-mars
Matin,"{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ...","{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ...","{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ...","{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ...","{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ..."
Aprem,"{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ...","{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ...","{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ...","{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ...","{'CONCEP.LOGICIELLE/BIG DATA': {'A': [], 'B': ..."


## On filtres les majeurs de chaque jours

In [9]:
import copy
def filtreMaj(pLstSemaine):
    Semaine =  {}
    lstMajeursinit = ['CONCEP.LOGICIELLE/BIG DATA','ROBOTIQUE','ELECTRONIQUE ET SYST EMB','INFRA DES RESEAUX','IMAGE','Pour tous']
    
    # creation dico {majeur: {grp: [], ...}, ...}
    dicMajinit = {}
    for maj in lstMajeursinit:
        dicMajinit[maj] = []    

    # parcours des jours de la semaine
    for jour in pLstSemaine :
        dicMaj = copy.deepcopy(dicMajinit)
        print(f'{jour["jour"]=}')
        Majeur = "Pour tous" #on initialise le majeur à tous
        Semaine[jour["jour"]] = {}

        # parcours de la matinée et on les ajoute au majeur correspondant 
        for Case in jour["Matin"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur].append(Case)
        Semaine[jour["jour"]]["Matin"] = copy.deepcopy(dicMaj)

        # parcours de l'aprem et on les ajoute au majeur correspondant
        dicMaj = copy.deepcopy(dicMajinit)
        Majeur = "Pour tous" #on initialise le majeur à tous
        for Case in jour["Aprem"]:
            if Case in lstMajeursinit:
                Majeur = Case
            dicMaj[Majeur].append(Case)
        Semaine[jour["jour"]]["Aprem"] = copy.deepcopy(dicMaj)

    print('fin')
    return Semaine

PlanningMajeur = filtreMaj(copy.deepcopy(LstSemaine)) # KK
df_planningMaj = pd.DataFrame(PlanningMajeur)
df_planningMaj

jour["jour"]='LUNDI-27-mars'
jour["jour"]='MARDI-28-mars'
jour["jour"]='MERCREDI-29-mars'
jour["jour"]='JEUDI-30-mars'
jour["jour"]='VENDREDI-31-mars'
fin


,LUNDI-27-mars,MARDI-28-mars,MERCREDI-29-mars,JEUDI-30-mars,VENDREDI-31-mars
Matin,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...
Aprem,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'..."


### Gestion des 4CGP

In [ ]:
def filtre4CGP(LstSemaine) : # KK
    Semaine =  {}
    for jour in LstSemaine :
        print(jour["jour"])
        Majeur = "Pour tous" #on initialise le majeur à tous
        Mat= {
            "Groupe A":[],
            "Groupe B":[],
            "Groupe C":[],
            "Chimie Médicinale et Hétérocycles":[],
            "Génie de la réaction chimique avancée":[],
            "Transition Energétique":[],
            "Analyse en ligne" : [],
            "Synthèse Macromoléculaire":[],
            "Pour tous":[]
        } 

filtre4CGP(LstSemaine)

In [5]:
with open(f'./Output_Json/Planning{date}.json', 'w+') as f:
    json.dump(PlanningMajeur, f, indent=4, cls=NumpyEncoder)

